# Assignment: Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`
### Do Q1 and Q2, and either Q3 or Q4, for a total of 3 questions.

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? 
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [20]:
import pandas as pd 
import numpy as np
import re 

#PART 1
AirBnBData = pd.read_csv('./data/airbnb_hw.csv') #read in the data 

print(AirBnBData['Price'].dtypes) #check the datatype of the column, want it to be int

def check_comma(value): #create a function that converts to numerics, checking if they have a comma 
    pattern = r'[0-9]+[,][0-9][0-9][0-9]' #regex pattern to match something formatted like 0,000
    if re.match(pattern, value): #checks if the string has a comma 
        cleaned = re.sub(r',', '', value) #substitutes the comma for a blank space
        cleaned = pd.to_numeric(cleaned) #converts to a numeric 
        return cleaned
    else:
        value = pd.to_numeric(value) #converts to numeric if there is no comma 
        return value
    
#apply the check_comma function to everything in the Price column and create a new column called PriceClean
AirBnBData['PriceClean'] = AirBnBData['Price'].apply(check_comma) 

print(AirBnBData['PriceClean'].dtypes) #check price after typecasting, should be int 

print(AirBnBData['PriceClean'].isna().sum().sum()) #check how many NA values have been created 

object
int64
0


In [22]:
#PART 2
PoliceData = pd.read_csv('./data/mn_police_use_of_force.csv')

print(PoliceData['subject_injury'].dtypes) #check type 

na_total = (PoliceData['subject_injury'].isna().sum())
total = (PoliceData['response_datetime'].count())
missing_injuries = na_total / total
print(missing_injuries) #around 76% of data is missing, which is concerning 

PoliceData['InjuryClean'] = PoliceData['subject_injury'] #create a new column for the cleaned data 
PoliceData['InjuryClean'] = PoliceData['subject_injury'].fillna('NA') #fill in the missing values with NA

#Crosstabulate the cleaned injury data with force type 
pd.crosstab(PoliceData['InjuryClean'], PoliceData['force_type'], rownames=['injury'], colnames=['force'])
#From this crosstabulation, we can see a lot of the missing data comes from instances where lots of force was
#used, such as Bodily Force, Chemical Irritant, Maximal Restraint Technique, Less Lethal, or Taser. This makes
#you wonder why the data is missing and if it could be from lack of response, not lack of injury. 

object
0.7619342359767892


force,Baton,Bodily Force,Chemical Irritant,Firearm,Gun Point Display,Improvised Weapon,Less Lethal,Less Lethal Projectile,Maximal Restraint Technique,Police K9 Bite,Taser
injury,,,,,,,,,,,
NA,2,7051,1421,0,27,74,87,0,170,31,985
No,0,1093,131,2,33,34,0,1,0,2,150
Yes,2,1286,41,0,44,40,0,2,0,44,172


In [23]:
#PART 3
PretrialData = pd.read_parquet('./data/justice_data.parquet')

#checks what the values are before cleaning 
print(PretrialData['WhetherDefendantWasReleasedPretrial'].unique())

#check the type of the data
print(PretrialData['WhetherDefendantWasReleasedPretrial'].dtypes)

#replaces all 9s with np.nan
PretrialData['WhetherDefendantWasReleasedPretrial'] = PretrialData['WhetherDefendantWasReleasedPretrial'].replace(9, np.nan)

#checks what the unique values are after replacement 
print(PretrialData['WhetherDefendantWasReleasedPretrial'].unique())

[9 0 1]
int64
[nan  0.  1.]


In [32]:
#PART 4
ImposedSentence = PretrialData['ImposedSentenceAllChargeInContactEvent']
SentenceType = PretrialData['SentenceTypeAllChargesAtConvictionInContactEvent']

print(ImposedSentence)
print(SentenceType)

#pd.crosstab(ImposedSentence, SentenceType, rownames=['imposed sentence'], colnames=['sentence type'])


0                        
1                      60
2                      12
3        .985626283367556
4                        
               ...       
22981                    
22982                    
22983                    
22984                    
22985                    
Name: ImposedSentenceAllChargeInContactEvent, Length: 22986, dtype: object
0        9
1        0
2        1
3        1
4        4
        ..
22981    4
22982    9
22983    4
22984    4
22985    4
Name: SentenceTypeAllChargesAtConvictionInContactEvent, Length: 22986, dtype: int64


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

In [43]:
SharkData = pd.read_excel('./data/GSAF5.xls', engine="xlrd")
print(SharkData.head(2))

SharkData = SharkData.drop('pdf', axis=1)
print(SharkData.head(2))


#print(SharkData.dtypes)

                  Date    Year        Type    Country              State  \
0  2025-01-23 00:00:00  2025.0  Unprovoked  Australia  Western Australia   
1  2025-01-22 00:00:00  2025.0  Unprovoked  Australia  Western Australia   

         Location Activity         Name Sex Age  ...       Species   \
0  Shipwreck Cove  Bathing         Boy    M  12  ...     Not stated   
1  Cheynes beach   Surfing  Dale Kittow   M  37  ...  Bronze whaler   

           Source  pdf href formula href Case Number Case Number.1  \
0  Bob Myatt GSAF  NaN          NaN  NaN         NaN           NaN   
1  Bob Myatt GSAF  NaN          NaN  NaN         NaN           NaN   

  original order Unnamed: 21 Unnamed: 22  
0            NaN         NaN         NaN  
1            NaN         NaN         NaN  

[2 rows x 23 columns]
                  Date    Year        Type    Country              State  \
0  2025-01-23 00:00:00  2025.0  Unprovoked  Australia  Western Australia   
1  2025-01-22 00:00:00  2025.0  Unprovoked

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q4.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?